We first launch a cluster and import it in the notebook, it should look like this :

from dask.distributed import Client

client = Client("tcp://127.0.0.1:46273")
client

In [2]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:43139")
client

<Client: 'tcp://127.0.0.1:43139' processes=4 threads=8, memory=29.40 GiB>

We load basic libraries (numpy, xarray, matplotlib, scipy) and one more specific :
    - xrft for Fourier transforms compatible with xarray : https://xrft.readthedocs.io/en/latest/


In [3]:
import numpy as np
import xarray as xr
import xrft
from scipy.interpolate import griddata
import matplotlib.colors as clr
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
store='s3://Pangeo/pangeo-forge/swot_adac/eNATL60/Region0{{ region }}/{{ datatype }}/{{ season }}.zarr'
ds = xr.open_dataset(store, engine='zarr', chunks={})
ds

AttributeError: module 'lib' has no attribute 'OpenSSL_add_all_algorithms'

In [7]:
import OpenSSL

AttributeError: module 'lib' has no attribute 'OpenSSL_add_all_algorithms'

In [4]:
from validate_catalog import all_params
params_dict, cat = all_params()

for entry in params_dict.keys():
    print(f"{entry} parameters and their allowable args are:")
    description = cat[entry].describe()
    params = description["user_parameters"]
    if len(params) != 0:
        for i in range(len(params)):
            print(f"""    {params[i]["name"]}: {params[i]["allowed"]}""")            
    else:
        print("    Not implemented.")
    print()

GIGATL parameters and their allowable args are:
    region: ['1', '2']
    datatype: ['surf', 'int']
    season: ['fma', 'aso']

HYCOM25 parameters and their allowable args are:
    region: ['1', '2', '3', '4', '5']
    datatype: ['surface_hourly']
    season: ['fma', 'aso']

HYCOM50 parameters and their allowable args are:
    region: ['1_GS', '2_GE', '3_MD']
    datatype: ['surf', 'int', 'int_wvel', 'grid']
    season: ['fma', 'aso', '']
    grid: ['grid', '']

eNATL60 parameters and their allowable args are:
    region: ['1', '2', '3']
    datatype: ['surface_hourly', 'interior_daily']
    season: ['fma', 'aso']

FESOM parameters and their allowable args are:
    datatype: ['surf', 'int']
    season: ['fma', 'aso']

ORCA36 parameters and their allowable args are:
    region: ['1', '2', '3', '4']
    datatype: ['surface_hourly', 'interior_daily', 'grid']
    season: ['fma', 'aso', '']
    grid: ['bathymetry', 'meshmask', '']

FIO-COM32 parameters and their allowable args are:
    reg

# Call data

In [5]:
ds = cat.eNATL60(region='1',datatype='surface_hourly', season='fma').to_dask()
ds

/srv/conda/envs/notebook/lib/python3.10/site-packages/xarray/backends/plugins.py:61: RuntimeWarning: Engine 'rasterio' loading failed:
module 'lib' has no attribute 'OpenSSL_add_all_algorithms'
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


AttributeError: module 'lib' has no attribute 'OpenSSL_add_all_algorithms'

# Interpolate finite-volume velocities to tracer points

In [ ]:
u = .5*(ds.sozocrtx.where(ds.umask!=0.) 
        + ds.sozocrtx.where(ds.umask!=0.).roll(x=-1)
       ).isel(x=slice(None,-1),y=slice(None,-1))
v = .5*(ds.somecrty.where(ds.vmask!=0.) 
        + ds.somecrty.where(ds.vmask!=0.).roll(y=-1)
       ).isel(y=slice(None,-1),x=slice(None,-1))
X = ds.e1t.cumsum('x').isel(y=slice(None,-1),x=slice(None,-1))
Y = ds.e2t.cumsum('y').isel(y=slice(None,-1),x=slice(None,-1))
X.plot()

# Interpolate onto uniform grid

In [ ]:
ny, nx = X.shape
X = X - X.isel(x=nx//2)
xx = (X - X.isel(y=-1,x=0)).values.flatten()
yy = (Y - Y.isel(y=0)).values.flatten()
dx = ds.e1t.min().values
dy = ds.e2t.min().values
A = (ds.e1t * ds.e2t).isel(y=slice(None,-1),x=slice(None,-1))

xxx, yyy = np.mgrid[0:nx*dx:dx, 0:ny*dy:dy]

In [ ]:
Aterp = griddata((xx, yy),
                 A.values.flatten(),
                 (xxx, yyy) 
                ).T 

In [ ]:
uterp = xr.DataArray(np.ones_like(u[::240]), dims=['time','YC','XC'],
                     coords={'time':ds.time_counter[::240].data,
                             'YC':np.arange(0,ny*dy,dy),
                             'XC':np.arange(0,nx*dx,dx)}
                    )
vterp = xr.DataArray(np.ones_like(v[::240]), dims=['time','YC','XC'],
                     coords={'time':ds.time_counter[::240].data,
                             'YC':np.arange(0,ny*dy,dy),
                             'XC':np.arange(0,nx*dx,dx)}
                    )

for it in range(len(ds.time_counter[::240])):
    uterp.isel(time=it)[:] = griddata((xx, yy),
                                      (u[::240]*A).isel(time_counter=it).values.flatten(),
                                      (xxx, yyy) 
                                     ).T 
    vterp.isel(time=it)[:] = griddata((xx, yy), 
                                      (v[::240]*A).isel(time_counter=it).values.flatten(),
                                      (xxx, yyy) 
                                     ).T
    
uterp /= Aterp
vterp /= Aterp

In [ ]:
fig, (ax1,ax2) = plt.subplots(nrows=1, ncols=2, figsize=(12,5))

im = ax1.pcolormesh((X - X.isel(y=-1,x=0)), (Y - Y.isel(y=0)),
                    u.isel(time_counter=0),
                    vmin=-2, vmax=2, cmap='RdBu_r')
uterp.isel(time=0).plot(ax=ax2, vmin=-2, vmax=2, cmap='RdBu_r')

fig.colorbar(im, ax=ax1)

# Compute isotropic power spectra

In [ ]:
Eu = xrft.isotropic_power_spectrum(uterp.fillna(0.), 
                                   dim=['YC','XC'], window='hann', detrend='linear', 
                                   true_phase=True, true_amplitude=True, 
                                   window_correction=True,
                                   truncate=True
                                  )
Ev = xrft.isotropic_power_spectrum(vterp.fillna(0.), 
                                   dim=['YC','XC'], window='hann', detrend='linear', 
                                   true_phase=True, true_amplitude=True,
                                   window_correction=True,
                                   truncate=True
                                  )

In [ ]:
fig, ax = plt.subplots()
fig.set_tight_layout(True)

ax.plot(Eu.freq_r*1e3, .5*(Eu+Ev).mean('time')*2*np.pi)

ax.set_xscale('log')
ax.set_yscale('log')
ax.grid(True)

ax.set_xlabel(r"[cpkm]", fontsize=12)
ax.set_ylabel(r"[(m$^2$ s$^{-2}$) / cpm]", fontsize=12)